# IND320 Course Project

In [ ]:
import pandas as pd

df = pd.read_csv('./assets/open-meteo-subset.csv')
print(df.head())


               time  temperature_2m (°C)  precipitation (mm)  \
0  2020-01-01T00:00                 -2.2                 0.1   
1  2020-01-01T01:00                 -2.2                 0.0   
2  2020-01-01T02:00                 -2.3                 0.0   
3  2020-01-01T03:00                 -2.3                 0.0   
4  2020-01-01T04:00                 -2.7                 0.0   

   wind_speed_10m (m/s)  wind_gusts_10m (m/s)  wind_direction_10m (°)  
0                   9.6                  21.3                     284  
1                  10.6                  23.0                     282  
2                  11.0                  23.5                     284  
3                  10.6                  23.3                     284  
4                  10.6                  22.8                     284  
